# NeoLook Scraper
* 2022.07.19a


## Step03: HTML 문서 분석하기

### 저장해 둔 Archive List 읽기

In [2]:
def readList(fileName):
    with open(fileName, 'r') as f: 
        lines = []  
        for line in f:
            #print(line)
            lines.append(line.rstrip())
        return lines
    print(len(lines)) 
       
list = readList("archivesList/total.txt")
print(len(list))
print(list[-10:])

47154
['20220629f', '20220630a', '20220630b', '20220630c', '20220630d', '20220630e', '20220630f', '20220630g', '20220630h', '20220630i']


In [8]:
def getHtmlText(fileName):
    dir = fileName[:4]
    file = fileName[4:]
    path = "htmls/" + dir + "/" + fileName + ".htm"
    print(path)
    with open(path, 'r', encoding='utf-8') as f: 
        return f.read()


text = getHtmlText(list[0])
#print(text)

#testList = ['20220602a', '20220602c', '20220603i', '20220601b'] #개인전, 그룹전, 그룹전, 저널

htmls/1989/19891106a.htm


In [71]:
from bs4 import BeautifulSoup as bs
import re
import random # debug 용

In [832]:
xtra = ["개인", "회화", "조각", "사진", "영상", "설치", "수묵", "서예", "드로잉", "판화", "도예"]
xtraDic = {"개인":"solo", "회화":"painting", "조각":"sculpture", "사진":"photography", "영상":"video", "설치":"installation", "수묵":"painting", "서예":"calligrapht", "드로잉":"drawing", "판화":"printing", "도예":"ceramic"}

def getTypeGenre(whichVolumePage):
    soup = bs(whichVolumePage, "html.parser")
    volumeTag = soup.find(class_="tag") #페이지 하단 하얀 박스에 표기된 정보
    volumeTag = volumeTag.contents[2].strip()
    print(volumeTag)

    type = ""
    genre = ""
    #if("展" in volumeTag):
    if(("展" in volumeTag or volumeTag[-1] == "전" or volumeTag[-1] == "제") and not ("조각회展" in volumeTag)):
        #volumeTag = volumeTag.split("/")
        if(len(volumeTag.split("/"))>1):
            type = "solo"
            #print("solo")
            genre = volumeTag.split("/")[-1].strip()            
        elif(len(volumeTag.split(" "))>=2 and (volumeTag.split(" ")[1][-1] == "展")):
            #print("xtra?")
            type = "group"
            #print("xtra group")
            genre = "group"           
            for item in xtra:
                last = volumeTag.split(" ")[-1]
                #print("item:", item, ", last:", last)
                if(item in last):
                    type = "solo"
                    #print("xtra solo")
                    #genre = volumeTag.split(" ")[1]
                    #genre = genre[:-1]
                    key = item
                    #print("key =", key)
                    genre = xtraDic[key]
                    break              
                '''
                else:
                    type = "group"
                    print("xtra group")
                    genre = "group"
                    break
                '''
        else:
            type = "group"
            genre = "group"
            print("group")
    else:
        print("NOT exhibition") 
        type = "NA" 
        genre = "NA"
    #print("type:", type)    
    #print("genre:", genre)
    return type, genre

htmlText = getHtmlText(list[random.randrange(0, 10000)])
type, genre = getTypeGenre(htmlText)
print("type =", type, ", genre =", genre)

htmls/2000/20001109a.htm
정규호展 / photography
type = solo , genre = photography


In [797]:
def getTitle(whichVolumePage):
    soup = bs(whichVolumePage, "html.parser")
    volumeTag = soup.find(class_="tag") #페이지 하단 하얀 박스에 표기된 정보
    volumeTag = volumeTag.contents[2].strip()
    print("volumeTag:", volumeTag)
    title = ""
    h1 = soup.find("h1")
    h2 = soup.find("h2")
    if(h1 != None):
        title = h1.text.strip()        
    else:
        title = h2.text.strip()
    print("title:",title)  
    return title

htmlText = getHtmlText(list[random.randrange(0, 10000)])
title = getTitle(htmlText)

htmls/2003/20030224a.htm
volumeTag: 영 페미니스트 미술가 연대 기획展
title: Anti Araki


In [807]:
#import re

def getSoloArtist(whichVolumePage):
    soup = bs(whichVolumePage, "html.parser")
    volumeTag = soup.find(class_="tag") #페이지 하단 하얀 박스에 표기된 정보
    volumeTag = volumeTag.contents[2].strip()
    print(volumeTag)

    artists = []
    tags = volumeTag.split("/")    
    if(len(tags) >= 2 and tags[0].strip()[-1] == "展"):
        print("parsed")
        artist = tags[0].strip()[:-1]
        artists.append(artist)
        #print(artists) 
    
    tags = volumeTag.split(" ")  
    if(len(tags) >= 2 and tags[-1][-1].strip() == "展"):
        artist = tags[0].strip()
        artists.append(artist)
        print(tags[-1]) 
    
    return artists

htmlText = getHtmlText(list[random.randrange(0, 10000)])
artists = getSoloArtist(htmlText)
print(artists)

htmls/2006/20060807d.htm
도영준 개인展
개인展
['도영준']


In [818]:
#import re

def getGroupArtist(whichVolumePage):
    soup = bs(whichVolumePage, "html.parser")
    volumeTag = soup.find(class_="tag") #페이지 하단 하얀 박스에 표기된 정보
    volumeTag = volumeTag.contents[2].strip()
    print(volumeTag)

    artists = []
    description = soup.find(class_="archives-description")
    dd = description.find_all("dd")
    #print(len(dd)) #캡션의 수 출력
    for child in dd:
        #print(child.text)
        if(len(child.text) > 1):
            artists.append(re.split(" |_", child.text.strip())[0])
    print(artists)
    return artists

htmlText = getHtmlText(list[random.randrange(0, 10000)])
artists = getGroupArtist(htmlText)

htmls/2008/20080325e.htm
내면풍경_innerscape展 / painting
['윤상윤', '박상희', '박상희', '윤상윤', '윤상윤']


In [340]:
#import re
extraGenre = ["개인", "화화", "조각", "목조각", "사진", "영상", "사진.영상", "설치영상"]

def getGenre(whichVolumePage):
    soup = bs(whichVolumePage, "html.parser")
    volumeTag = soup.find(class_="tag") #페이지 하단 하얀 박스에 표기된 정보
    volumeTag = volumeTag.contents[2].strip()
    print(volumeTag)

    genre = ""
    if("展" in volumeTag or volumeTag[-1] == "전"):
        #isExhibition = True
        #print("exhibition")
        tag = volumeTag.split("/")
        #tag = re.split(" |/", volumeTag.strip())
        if(len(tag)>1):
            genre = tag[-1].strip()            
        else:
            #print(tag.contents[2].strip())
            for item in extraGenre:
                #print(item)
                
                if(item in volumeTag):
                    print("extra:", item)
                    genre = item 
                #break                             
    else:
        genre = "NA" 
        
    if("공모" in volumeTag):
        genre = "NA"
    print(genre)
    return genre


htmlText = getHtmlText(list[random.randrange(0, 10000)])
genre = getGenre(htmlText)

htmls/2006/20061203a.htm
차혜림 회화展



In [ ]:
import re

def getGenre(whichVolumePage):
    soup = bs(whichVolumePage, "html.parser")
    volumeTag = soup.find(class_="tag") #페이지 하단 하얀 박스에 표기된 정보
    #result = result.div.h2.span.text
    volumeTag = volumeTag.contents[2].strip()
    print(volumeTag)

    type = ""
    title = ""
    genre = ""
    if("展" in volumeTag):
        #isExhibition = True
        #print("exhibition")
        volumeTag = volumeTag.split("/")
        if(len(volumeTag)>1):
            type = "solo"
            print("solo")
            genre = volumeTag[-1].strip()            
        else:
            type = "group"
            genre = "group"
            print("group")
    else:
        print("NOT exhibition") 
        type = "etc" 
    print(genre)

    artists = []
    if(type == "solo"):
        artist = volumeTag[0].strip()
        artists.append(artist[:-1])
        print(artists)
    elif(type == "group"):
        description = soup.find(class_="archives-description")
        dd = description.find_all("dd")
        print(len(dd)) #캡션의 수 출력
        for child in dd:
            #print(child.text.strip().split("_")[0])
            #print(re.split(" |_", child.text.strip())[0])
            artists.append(re.split(" |_", child.text.strip())[0])
        print(artists)

    return type, genre, artists

testList = ['19891106b','20000120a','19991207a','19991005a','19970303a','19931126a','19930209a','19921013a','19921002a','19920901b', '19920505a', '19910309a', '20220602a', '20220602c', '20220603i', '20220601b'] #개인전, 그룹전, 그룹전, 저널
htmlText = getHtmlText("htmls/"+testList[0][:4]+"/"+testList[0]+".htm")
#print(htmlText)
type, genre, artists = getType(htmlText)
print("")

In [16]:
htmlText = getHtmlText("htmls/1989/19891106a.htm")
#print(htmlText)
type = getType(htmlText)




<span class="line">
          오윤_김진하_이인철_이상호展
         </span>


<span class="line">
</span>


<span class="line">
          1989_1106 ▶ 1989_1208
         </span>




<h2>
<span class="line">
          오윤_김진하_이인철_이상호展
         </span>
<span class="line">
</span>
<span class="line">
          1989_1106 ▶ 1989_1208
         </span>
</h2>